## Merge and Concatenate

#### Merging basics

In [1]:
import pandas as pd

In [2]:
pops = pd.DataFrame({'Country Code': ['THA', 'USA', 'NLD', 'GER'],
                     'Population': [68, 328, 17, 82]})

capitals = pd.DataFrame({'Country Code': ['THA', 'USA', 'NLD', 'GER', 'JPN'],
                         'Country Name': ['Thailand', 'United States', 'Netherlands', 'Germany', 'Japan'],
                         'Capital Name': ['Bangkok', 'Washington D.C.', 'Amsterdam', 'Berlin', 'Tokyo']})
display(pops, capitals)

,Country Code,Population
0,THA,68
1,USA,328
2,NLD,17
3,GER,82


,Country Code,Country Name,Capital Name
0,THA,Thailand,Bangkok
1,USA,United States,Washington D.C.
2,NLD,Netherlands,Amsterdam
3,GER,Germany,Berlin
4,JPN,Japan,Tokyo


In [3]:
# merge example - inner join
pd.merge(capitals, pops, 
         on='Country Code', how='inner')

,Country Code,Country Name,Capital Name,Population
0,THA,Thailand,Bangkok,68
1,USA,United States,Washington D.C.,328
2,NLD,Netherlands,Amsterdam,17
3,GER,Germany,Berlin,82


In [4]:
# merge example - outer join
pd.merge(capitals, pops, 
         on='Country Code', how='outer')

,Country Code,Country Name,Capital Name,Population
0,THA,Thailand,Bangkok,68.0
1,USA,United States,Washington D.C.,328.0
2,NLD,Netherlands,Amsterdam,17.0
3,GER,Germany,Berlin,82.0
4,JPN,Japan,Tokyo,NaN


In [5]:
# merge example - left join
pd.merge(capitals, pops, 
         on='Country Code', how='left')

,Country Code,Country Name,Capital Name,Population
0,THA,Thailand,Bangkok,68.0
1,USA,United States,Washington D.C.,328.0
2,NLD,Netherlands,Amsterdam,17.0
3,GER,Germany,Berlin,82.0
4,JPN,Japan,Tokyo,NaN


In [6]:
# merge example - right join
pd.merge(capitals, pops, 
         on='Country Code', how='right')

,Country Code,Country Name,Capital Name,Population
0,THA,Thailand,Bangkok,68
1,USA,United States,Washington D.C.,328
2,NLD,Netherlands,Amsterdam,17
3,GER,Germany,Berlin,82


#### Concatenate basics

In [7]:
pops2 = pd.DataFrame({'Country Code': ['KOR', 'AFG', 'AUT'],
          'Population': [8, 29, 8]})
display(pops2)

,Country Code,Population
0,KOR,8
1,AFG,29
2,AUT,8


In [8]:
# pops first
pd.concat([pops, pops2], axis=0).reset_index(drop=True)

,Country Code,Population
0,THA,68
1,USA,328
2,NLD,17
3,GER,82
4,KOR,8
5,AFG,29
6,AUT,8


In [9]:
# pops second
pd.concat([pops2, pops])

,Country Code,Population
0,KOR,8
1,AFG,29
2,AUT,8
0,THA,68
1,USA,328
2,NLD,17
3,GER,82


In [10]:
# append
pops = pops.append(pops2)
pops.reset_index(drop=True)

,Country Code,Population
0,THA,68
1,USA,328
2,NLD,17
3,GER,82
4,KOR,8
5,AFG,29
6,AUT,8


#### Mapping tables example: Toys dataset

In [11]:
products = pd.read_csv('../datasets/toys/products.csv')
stores = pd.read_csv('../datasets/toys/stores.csv')
sales = pd.read_csv('../datasets/toys/sales.csv')
inventory = pd.read_csv('../datasets/toys/inventory.csv')

print('Products: ', products.shape)
print('Stores: ', stores.shape)
print('Sales: ', sales.shape)
print('Inventory: ', inventory.shape)

Products:  (35, 5)
Stores:  (50, 5)
Sales:  (829262, 5)
Inventory:  (1593, 3)


In [12]:
products.head(3)

,Product_ID,Product_Name,Product_Category,Product_Cost,Product_Price
0,1,Action Figure,Toys,$9.99,$15.99
1,2,Animal Figures,Toys,$9.99,$12.99
2,3,Barrel O' Slime,Art & Crafts,$1.99,$3.99


In [13]:
stores.head(3)

,Store_ID,Store_Name,Store_City,Store_Location,Store_Open_Date
0,1,Maven Toys Guadalajara 1,Guadalajara,Residential,1992-09-18
1,2,Maven Toys Monterrey 1,Monterrey,Residential,1995-04-27
2,3,Maven Toys Guadalajara 2,Guadalajara,Commercial,1999-12-27


In [14]:
sales.head(3)

,Sale_ID,Date,Store_ID,Product_ID,Units
0,1,2017-01-01,24,4,1
1,2,2017-01-01,28,1,1
2,3,2017-01-01,6,8,1


In [15]:
inventory.head(3)

,Store_ID,Product_ID,Stock_On_Hand
0,1,1,27
1,1,2,0
2,1,3,32


In [16]:
# map store names using store id
mapped = pd.merge(sales, stores[['Store_ID', 'Store_Name', 'Store_Location']],
                  on='Store_ID', how='left')
mapped.head(3)

,Sale_ID,Date,Store_ID,Product_ID,Units,Store_Name,Store_Location
0,1,2017-01-01,24,4,1,Maven Toys Aguascalientes 1,Downtown
1,2,2017-01-01,28,1,1,Maven Toys Puebla 2,Downtown
2,3,2017-01-01,6,8,1,Maven Toys Mexicali 1,Commercial


In [17]:
# map product names and prices using product id 
mapped2 = pd.merge(mapped, products[['Product_ID', 'Product_Name', 'Product_Price']],
                   on='Product_ID', how='left')
mapped2.head(3)

,Sale_ID,Date,Store_ID,Product_ID,Units,Store_Name,Store_Location,Product_Name,Product_Price
0,1,2017-01-01,24,4,1,Maven Toys Aguascalientes 1,Downtown,Chutes & Ladders,$12.99
1,2,2017-01-01,28,1,1,Maven Toys Puebla 2,Downtown,Action Figure,$15.99
2,3,2017-01-01,6,8,1,Maven Toys Mexicali 1,Commercial,Deck Of Cards,$6.99


In [18]:
mapped2['Product_Price'] = mapped2['Product_Price'].str.replace("$","").astype(float)
mapped2['Sales_Amount'] = mapped2['Units']*mapped2['Product_Price']

In [19]:
mapped2[mapped2['Units']>1].head(10)

,Sale_ID,Date,Store_ID,Product_ID,Units,Store_Name,Store_Location,Product_Name,Product_Price,Sales_Amount
12,13,2017-01-01,37,8,2,Maven Toys Ciudad de Mexico 3,Residential,Deck Of Cards,6.99,13.98
19,20,2017-01-01,3,24,2,Maven Toys Guadalajara 2,Commercial,Nerf Gun,19.99,39.98
29,30,2017-01-01,34,18,3,Maven Toys Villahermosa 1,Downtown,Lego Bricks,39.99,119.97
34,35,2017-01-01,39,34,3,Maven Toys Xalapa 2,Downtown,Toy Robot,25.99,77.97
37,38,2017-01-01,32,8,4,Maven Toys Hermosillo 1,Residential,Deck Of Cards,6.99,27.96
44,45,2017-01-01,41,8,3,Maven Toys Hermosillo 2,Downtown,Deck Of Cards,6.99,20.97
46,47,2017-01-01,30,1,3,Maven Toys Guadalajara 3,Airport,Action Figure,15.99,47.97
55,56,2017-01-01,41,8,2,Maven Toys Hermosillo 2,Downtown,Deck Of Cards,6.99,13.98
63,64,2017-01-01,34,18,2,Maven Toys Villahermosa 1,Downtown,Lego Bricks,39.99,79.98
71,72,2017-01-01,19,6,2,Maven Toys Puebla 1,Commercial,Colorbuds,14.99,29.98


In [20]:
# filter date to be oct 1, 2018
oct_sales = mapped2[mapped2['Date']=='2018-09-30']
oct_sales.shape

(2010, 10)

In [21]:
# aggregate sales
oct_sales_sum = oct_sales.groupby(['Store_ID', 'Product_ID']).Units.sum().reset_index()
oct_sales_sum.head()

,Store_ID,Product_ID,Units
0,1,5,5
1,1,9,35
2,1,12,4
3,1,19,3
4,1,24,4


In [22]:
# map inventory
oct_inv = pd.merge(oct_sales_sum, inventory,
                   on=['Store_ID', 'Product_ID'], 
                   how='left')
oct_inv.tail()

,Store_ID,Product_ID,Units,Stock_On_Hand
198,49,3,20,14
199,49,10,3,36
200,49,24,8,23
201,49,25,4,23
202,49,30,2,3


In [23]:
oct_inv['Stock_Gap'] = oct_inv['Stock_On_Hand'] - oct_inv['Units']
oct_inv['Stock_Gap'].describe()

count    203.000000
mean       6.714286
std       24.014376
min      -67.000000
25%       -6.000000
50%        3.000000
75%       17.000000
max       98.000000
Name: Stock_Gap, dtype: float64